In [1]:
#LIBRARIES
import pandas as pd
import re
import numpy as np

## nomalice parameters
from unidecode import unidecode

import spacy  # For preprocessing
from scipy import spatial
from time import time  # To time our operations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

from tqdm import tqdm

from deep_translator import GoogleTranslator
nlp = spacy.load("en_core_web_sm")

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
## nomalice parameters
from unidecode import unidecode

C:\Users\War\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_excel("Profiles required for a representative sample of 600_.xlsx")

In [3]:
#MINUSCULA

df["dict"] =  df["dict"].str.lower()
df["Bootcamps_unif"] =  df["Bootcamps_unif"].str.lower()

In [4]:
#FUNCIÓN PARA TRADUCIR

def translateString(to_translate, languageFinal):
    translated = GoogleTranslator(source='auto', target=languageFinal).translate(to_translate)
    return translated

#FUNCIÓN PARA LIMPIAR
nlp = spacy.load("en_core_web_sm")

def cleaning(text):
    ##text = text.lower()
    #text = text.replace('[>/,\.!?\-!?\n\)\(\r]', ' ') 
    text = re.sub("[^A-Za-z']+", ' ', str(text))
    doc = nlp(text)
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    #txt = [token.lemma_ for token in doc if not token.is_stop]
    txt = [tok.lemma_.lower() for tok in doc
                if not tok.is_stop and not tok.is_punct and tok.is_alpha ]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    ## return an array of ewords
    ##return txt
    
    rta = ""
    for t in txt:
        print(t)
        rta+= " "+t
    return rta

# HOMOGENIZACIÓN INICIAL (VOLVER A LA ESTRUCTURA INICIAL)

In [5]:

#FUNCIÓN QUE CAPTURA HETEROGENEIDADES QUE HUBO EN LA RECOLECCIÓN DE DATOS

def homo(q):
    
    q = q.replace("\n", " ") #SALTOS
    q = " ".join(q.split()) #ESPACIOS
    q = q.replace("<!---->", " ")

    H = re.findall("[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]", q) #20162020
    for i in H: 
        q = q.replace(i, f"'start': '{i[0:4]}', 'end': '{i[4:8]}'")
    
    K = re.findall('<span class="pv-entity__secondary-title separator">[^0-9]*</span>', q) # '<span class="pv-entity__secondary-title separator">Jornada completa</span>'
    for i in K:
        q = q.replace(i, "")
        
    Q = re.findall('[A-z][A-z][A-z] [0-9][0-9][0-9][0-9]', q) #SEP 2020
    for i in Q:
        q = q.replace(i, f"{i[4:8]}")

    educ = str(q.split("}],")[0]+"}]")
    job_title = str(q.split("}],'")[1].split("',[{'")[0])
    if "job_title" in q:
        job_history = str("[{'"+q.split("',[{'")[1])
    else:
        job_history = ""

    return educ, job_title, job_history



In [6]:
#LOOP PARA EDUCACIÓN

df["educ"] = np.nan;
df["dict"] = df["dict"].replace(np.nan, "")

for i in tqdm(range(len(df))):
    try:
        if len(df["dict"][i])>0:
            df["educ"][i] = homo(df["dict"][i])[0]
    except:
        None

  0%|          | 0/649 [00:00<?, ?it/s]<ipython-input-6-b46f5e288b41>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["educ"][i] = homo(df["dict"][i])[0]
C:\Users\War\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 649/649 [00:00<00:00, 3488.98it/s]


In [7]:
#LOOP PARA JOB TITLE (este en realidad es el encabezado, hubo gente que no tenía trabajo: puede no coincidir con el trabajo pero sirve para la gente que no tiene historial)

df["job_title"] = np.nan;
df["dict"] = df["dict"].replace(np.nan, "")

for i in tqdm(range(len(df))):
    try:
        if len(df["dict"][i])>0:
            df["job_title"][i] = homo(df["dict"][i])[1]
    except:
        None

  0%|          | 0/649 [00:00<?, ?it/s]<ipython-input-7-7f3ed8e1b60a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["job_title"][i] = homo(df["dict"][i])[1]
100%|██████████| 649/649 [00:00<00:00, 2309.45it/s]


In [8]:
#LOOP PARA HOB HISTORY

df["job_history"] = np.nan;
df["dict"] = df["dict"].replace(np.nan, "")

for i in tqdm(range(len(df))):
    try:
        if len(df["dict"][i])>0:
            df["job_history"][i] = homo(df["dict"][i])[2]
    except:
        None

  0%|          | 0/649 [00:00<?, ?it/s]<ipython-input-8-797f3fa8e5ba>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["job_history"][i] = homo(df["dict"][i])[2]
100%|██████████| 649/649 [00:00<00:00, 3228.65it/s]


# PARTIR DE LOS ALGORITMOS PREVIOS: OBTENER EDUCACION, HISTORIAL LABORAL Y TITULO ACTUAL HOMOGENIZADO Y TRADUCIDO

In [9]:
#HOMOGENIZACIÓN DE EDUCACION

L = []

for i in df["educ"]:
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]


K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO

for individuo in tqdm(L): #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'education_org': '.*?'", ins)) ) > 0:
            c["education"] =  re.findall("'education_org': '.*?'", ins)[0].replace("'education_org': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall("'degrees': '.*?'", ins)) ) > 0:
            c["degrees"] =  re.findall("'degrees': '.*?'", ins)[0].replace("'degrees': ", "").replace("'", "")

        #EXTRAER SUMMARY
        if ( len(re.findall("'summary': '.*?'", ins)) ) > 0:
            c["summary"] =  re.findall("'summary': '.*?'", ins)[0].replace("'summary': ", "").replace("'", "")


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            c["start"] = (c["start"])

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            c["end"] = (c["end"])

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

df["educ_history_clean"] = K


100%|██████████| 649/649 [00:00<00:00, 6300.63it/s]


In [10]:


# for i in tqdm(K):
#     for e in tqdm(i):
#         if "degrees" in e and len(e["degrees"])>0:
#            try:
#                e["degrees"] = translateString(str(e["degrees"]), "en")
#                e["degrees"] = cleaning(e["degrees"])
#            except:
#               None

#         if "education" in tqdm(e) and len(e["education"])>0:
#            try:
#             e["education"] = translateString(str(e["education"]), "en")
#             e["education"] = cleaning(e["education"])
#            except:
#               None

# df["educ_history_clean"] = K

In [11]:
#HOMOGENIZAR HISTORIAL

#SEPARANDO LOS STRINGS DE CADA TRABAJO

L = []

for i in df.job_history:
    l = str(i).split("},") #convertir en string el job history del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for job in l: #para cada trabajo
        job = job.replace("[", "").replace("]", "") + "}" #separelos y conviertalas en diccionarios
        k.append(job) #añadalas a la lista de jobs
    L.append(k) 

#EXTRAER DE MANERA ORDENADA EL JOB HISTORY, LA FECHA DE TERMINACIÓN Y COMIENZO

K = [] 

for individuo in tqdm(L): 

    educacion_ind = [] 

    for ins in individuo: 
        ins = ins.replace('"', "'")

        c = {}

        #EXTRAER JOB-TITLE
        if ( len(re.findall("'job_title': '.*?'", ins)) ) > 0:
            c["job_title"] =  re.findall("'job_title': '.*?'", ins)[0].replace("'job_title': ", "").replace("'", "")
            c["job_title"] = str(c["job_title"])
            # c["job_title"] = cleaning(c["job_title"])

        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            c["start"] = str((c["start"][::-1][0:4][::-1])) #SOLO CONSERVAR EL AÑO 

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            c["end"] = str((c["end"][::-1][0:4][::-1])) #SOLO CONSERVAR EL AÑO

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general


df["job_history_clean"] = K



100%|██████████| 649/649 [00:00<00:00, 4972.56it/s]


In [12]:
#TRADUCIR Y LIMPIAR 

# for i in tqdm(K[:]):
#     for e in tqdm(i):
#         # A = "job_title" in e
#         # if len(e["job_title"])>0 and A == True:
#         try:
#             e["job_title"] = translateString(str(e["job_title"]), "en")
#             e["job_title"] = cleaning(e["job_title"])
#         except:
#             None


# df["job_history_clean"] = K

# ENCONTRAR EL AÑO DE GRADUACIÓN

In [13]:
# ENCONTRAR MATCH

df["bootcamp_educ"] = np.nan
df["ratio"] = np.nan

for i in range(len(df)):
    for h in range(len(df.educ_history_clean[i])):
        if "education" in df.educ_history_clean[i][h]:

            wordLen = len(df.educ_history_clean[i][h]["education"])
            ratio = fuzz.partial_ratio(df.Bootcamps_unif[i], df.educ_history_clean[i][h]["education"])
            if ratio>=85:
                df["bootcamp_educ"][i] = df.educ_history_clean[i][h]["education"]
                df["ratio"][i] = ratio

            if  df.educ_history_clean[i][h]["education"] == "hola&lt;​code/&gt;":
                df["bootcamp_educ"][i] = df.educ_history_clean[i][h]["education"]
                df["ratio"][i] = ratio
            
            if  df.educ_history_clean[i][h]["education"] == "dev.f.":
                df["bootcamp_educ"][i] = df.educ_history_clean[i][h]["education"]
                df["ratio"][i] = ratio

dfk = df[df["bootcamp_educ"].notna()] #40 quedan sin hacer match


<ipython-input-13-b08a608edb3b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bootcamp_educ"][i] = df.educ_history_clean[i][h]["education"]
<ipython-input-13-b08a608edb3b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ratio"][i] = ratio
<ipython-input-13-b08a608edb3b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bootcamp_educ"][i] = df.educ_history_clean[i][h]["education"]
<ipython-input-13-b08a608edb3b>:18: Se

In [14]:
dfna = df[df["bootcamp_educ"].isna()]
dfna.to_excel("base_con_nas.xlsx", index = False)

In [15]:
dfk

,Bootcamps,Bootcamps_unif,Link,dict,educ,job_title,job_history,educ_history_clean,job_history_clean,bootcamp_educ,ratio
0,Digital House Argentina,digital house,https://www.linkedin.com/in/simontoyabe/,[{'education_org': 'universidad pontificia bol...,[{'education_org': 'universidad pontificia bol...,excited to learn new things and to transform ...,"[{'job_title': 'process improvement analyst', ...",[{'education': 'universidad pontificia bolivar...,"[{'job_title': 'process improvement analyst', ...",digital house,100.0
1,Digital House Argentina,digital house,https://www.linkedin.com/in/carolinahernandezl...,"[{'education_org': 'universidad el bosque','de...","[{'education_org': 'universidad el bosque','de...",jefe de gestion humana en kjiplas s.a,"[{'job_title': 'jefe de gestión humana', 'comp...","[{'education': 'universidad el bosque', 'degre...","[{'job_title': 'jefe de gestión humana', 'star...",digital house argentina,100.0
2,Digital House Argentina,digital house,https://www.linkedin.com/in/sabrinaruggiero/,[{'education_org': 'universidad de ciencias em...,[{'education_org': 'universidad de ciencias em...,marketing head north latam at jokr,"[{'job_title': 'marketing head north latam', '...",[{'education': 'universidad de ciencias empres...,"[{'job_title': 'marketing head north latam', '...",digital house,100.0
3,Digital House Argentina,digital house,https://www.linkedin.com/in/laura-constanza-pr...,"[{'education_org': 'digital house','degrees': ...","[{'education_org': 'digital house','degrees': ...",gerente de proyectos en méntor,"[{'job_title': 'gerente de proyectos', 'compan...","[{'education': 'digital house', 'degrees': 'an...","[{'job_title': 'gerente de proyectos', 'start'...",digital house,100.0
4,Digital House Argentina,digital house,https://www.linkedin.com/in/paola-mejia-valder...,"[{'education_org': 'digital house','degrees': ...","[{'education_org': 'digital house','degrees': ...",business intelligence manager at gsk,[{'job_title': 'business intelligence manager'...,"[{'education': 'digital house', 'degrees': 'da...",[{'job_title': 'business intelligence manager'...,digital house,100.0
...,...,...,...,...,...,...,...,...,...,...,...
644,Le Wagon,le wagon,https://www.linkedin.com/in/danielrpo/?lipi=ur...,"[{'education_org': 'le wagon','degrees': ['ful...","[{'education_org': 'le wagon','degrees': ['ful...",seo manager - performance analyst,"[{'job_title': 'content team lead latam', 'com...","[{'education': 'le wagon', 'degrees': 'full st...","[{'job_title': 'content team lead latam', 'sta...",le wagon,100.0
645,Plataforma 5,plataforma 5,https://www.linkedin.com/in/karen-martinez-9ab...,[{'education_org': 'plataforma 5 - coding boot...,[{'education_org': 'plataforma 5 - coding boot...,software developer - freelance,[{'job_title': 'full stack developer &amp;&amp...,[{'education': 'plataforma 5 - coding bootcamp...,[{'job_title': 'full stack developer &amp;&amp...,plataforma 5 - coding bootcamp,100.0
646,Hack Academy,hack academy,https://www.linkedin.com/in/macarena-clark-seg...,[{'education_org': 'universidad de la empresa'...,[{'education_org': 'universidad de la empresa'...,project manager en fenicio ecommerce,"[{'job_title': 'project manager', 'company_nam...","[{'education': 'universidad de la empresa', 'd...","[{'job_title': 'project manager', 'start': '20...",hack academy,100.0
647,MindHub,mindhub,https://www.linkedin.com/in/cristianmachado121...,[{'education_org': 'universidad tecnológica na...,[{'education_org': 'universidad tecnológica na...,desarrollador full stack jr,"[{'job_title': 'analista funcional', 'company_...",[{'education': 'universidad tecnológica nacion...,"[{'job_title': 'analista funcional', 'start': ...",mindhub la,100.0


In [16]:
dfk["bootcamp_educ"].unique()

array(['digital house', 'digital house argentina', 'digital house ',
       'digital house brasil', 'digital house brazil', 'acámica',
       'acamica', 'acámica (globant - ibm)', 'laboratoria', 'ironhack',
       'holberton school', 'desafío latam', 'ada itw', 'ada',
       'hola&lt;\u200bcode/&gt;', 'hola', 'dev.f.', 'le wagon',
       'plataforma 5 - coding bootcamp', 'hack academy', 'hack academy ',
       'mindhub la', '4geeks academy', 'bogotá dev', 'muktek academy',
       'acámica - globant'], dtype=object)

In [17]:
dfk = dfk.reset_index(drop = True)

dfk["end_boot"] = np.nan 
dfk["end_boot_cat"] = np.nan

c = 0

for i in range(len(dfk.educ_history_clean)):
    for e in range(len(dfk.educ_history_clean[i])): #DENTRO DE LA EDUCACIÓN DE CADA INDIVIDUO
        if "education" in dfk.educ_history_clean[i][e].keys() and dfk.educ_history_clean[i][e]["education"] == dfk.bootcamp_educ[i]:

            if "end" in dfk.educ_history_clean[i][e].keys():
                dfk["end_boot"][i] = dfk.educ_history_clean[i][e]["end"]
                dfk["end_boot_cat"][i] = "end"
                
            elif "start" in dfk.educ_history_clean[i][e].keys():
                dfk["end_boot"][i] = dfk.educ_history_clean[i][e]["start"] #SI NO TENÍA EL FINAL POR LO MENOS SE LE IMPUTÓ EL INICIAL, EXISTE LA POSIBILIDAD DE QUE NO LO HAYA TERMINADO
                dfk["end_boot_cat"][i] = "start"



for i in range(len(dfk.end_boot)):
    if dfk.end_boot[i] == "":
        dfk.end_boot[i] = 2020
        dfk.end_boot_cat[i] = "inp"


dfk["end_boot"][0]

<ipython-input-17-835df4ac2b73>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfk["end_boot"][i] = dfk.educ_history_clean[i][e]["end"]
<ipython-input-17-835df4ac2b73>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfk["end_boot_cat"][i] = "end"
C:\Users\War\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-17-835df4ac2b

2023.0

In [18]:
dfk.to_excel("1_data_base.xlsx", index = False)

# UNIR  LAS DOS BASES

In [70]:

dfi = pd.read_excel("base_inicial.xlsx")

In [71]:
df1 = dfi[["bootcamp", "linkedin_profile_url", "job_title", "education", "job_history_clean", "end_boot", "end_boot_cat"]]
df1.columns = ["Bootcamps_unif", "Link", "job_title", "educ_history_clean", "job_history_clean", "end_boot", "end_boot_cat"]

In [72]:
df1["etapa"] = 1;

<ipython-input-72-fc040e159e59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["etapa"] = 1;


In [73]:
df2 = dfk [["Bootcamps_unif", "Link", "job_title", "educ_history_clean", "job_history_clean", "end_boot", "end_boot_cat" ]]
df2["etapa"]=2;

df2.end_boot = df2.end_boot.replace(np.nan, 2020)
df2.end_boot_cat = df2.end_boot_cat.replace(np.nan, "inp_2020")

<ipython-input-73-a2391ab1d9e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["etapa"]=2;
C:\Users\War\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [74]:
df3 = df1.append(df2)

In [75]:
df3.isna().sum()


Bootcamps_unif         0
Link                   0
job_title             11
educ_history_clean     3
job_history_clean     31
end_boot               0
end_boot_cat           0
etapa                  0
dtype: int64

In [76]:
df3.to_excel("db_vf.xlsx", index = False)